In [6]:
from flask import Flask, render_template, request, redirect, url_for, flash
import proyecto
import os
import shutil

este codigo valida archivos excel

In [ ]:


app = Flask(__name__)  # Inicializa la aplicación Flask
app.secret_key = 'supersecretkey'  # Clave secreta para manejar sesiones y mensajes flash

UPLOAD_FOLDER = 'uploads'  # Carpeta para subir archivos
VALIDATED_FOLDER = 'validated'  # Carpeta para archivos validados

# Crea las carpetas si no existen
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

if not os.path.exists(VALIDATED_FOLDER):
    os.makedirs(VALIDATED_FOLDER)

@app.route('/')
def principal():
    # Renderiza la plantilla principal
    return render_template('index.html')

@app.route('/validador', methods=['POST'])
def validador():
    # Verifica si los archivos están en la solicitud
    if 'file_excel' not in request.files or 'file_json' not in request.files:
        flash('No file part')
        return redirect(url_for('principal'))
    
    file_excel = request.files['file_excel']
    file_json = request.files['file_json']
    
    # Verifica si los archivos tienen nombre
    if file_excel.filename == '' or file_json.filename == '':
        flash('No selected file')
        return redirect(url_for('principal'))
    
    if file_excel and file_json:
        # Guarda los archivos subidos
        excel_path = os.path.join(UPLOAD_FOLDER, file_excel.filename)
        json_path = os.path.join(UPLOAD_FOLDER, file_json.filename)
        
        file_excel.save(excel_path)
        file_json.save(json_path)
        
        # Valida el archivo Excel con el archivo JSON usando la función del proyecto
        resultado = proyecto.validar_excel_con_cerberus(excel_path, json_path)
        
        flash(resultado['message'])  # Muestra el mensaje de resultado
        
        if resultado['status'] == 'success':
            # Copia el archivo validado a la carpeta de validados
            validated_excel_path = os.path.join(VALIDATED_FOLDER, file_excel.filename)
            shutil.copy(excel_path, validated_excel_path)
            flash(f'Archivo validado y copiado a {validated_excel_path}')
        
        return redirect(url_for('principal'))

if __name__ == '__main__':
    # Ejecuta la aplicación en modo debug en el puerto 3306
    app.run(debug=True, port=3306)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:3306
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\hectord.godoy\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Enviar correo

In [ ]:
def enviar_correo_con_errores(destinatario, archivo_adj):
    email_remitente = "jnsebas.barrios@gmail.com"  # Cambia esto a tu correo
    email_password = "TU_CONTRASEÑA_APP"  # Usa una App Password en Gmail

    #asunto = "Errores en Validación de Excel"
    cuerpo = "Adjunto encontrarás los errores de validación del archivo Excel."

    # Crear el mensaje de correo
    mensaje = EmailMessage()
    mensaje["From"] = email_remitente
    mensaje["To"] = destinatario
    mensaje["Subject"] = asunto
    mensaje.set_content(cuerpo)